# Context Managers
Consider the following




In [ ]:
f = open('foo.txt','w')
for x in range(10000):
    f.write(str(x)+'\n')
f.close()

why do we need to close, what happens if we forget? What if loop throws an exception

In [ ]:
f = open('foo.txt','w')
try:
    for x in range(10000):
        f.write(str(x)+'\n')
except:
    raise
finally:
    f.close()


Often we have pairs of things that have to occur open/close, start/stop, etc and you want to guarantee that these pair-wise opeartions always occur

That's where the context managers comes in.

General form of usage is:
```
with cm_entity as resource:
   do_stuff with resource
```

In [ ]:
with open('foo.txt','w') as f:
    for x in range(10000):
        f.write(str(x)+'\n')

## So what...
So if we don't close a file then things mostly work out right, eventually memory gets reclaimed or not, and there is a lot of memory...

But let's consider another example



In [ ]:
from threading import Thread,Lock

my_lock = Lock()
def my_function(n,m):    
    my_lock.acquire()
    # Pretend this is something we need to ensure only single access to so we need a lock
    r = n/m
    print("Result is %s\n"%r)
    my_lock.release()

def launch_thread(n,m):
    t = Thread(target = my_function, args = (n,m))
    t.start()
    return t
try:
    t1 = launch_thread(2,4)
    t2 = launch_thread(3,6)
    t3 = launch_thread(5,2)
    t1.join()
    t2.join()
    t3.join()
finally:
    print("done")


So we handled it fine. Even with a nice try block to handle any exceptions...

But what happens here

In [ ]:
try:
    t1 = launch_thread(2,4)
    t2 = launch_thread(3,0)
    t3 = launch_thread(5,2)
    t1.join()
    t2.join()
    t3.join()
finally:
    print("done")

Let's compare using a context handler now (Lock provides for one)

In [ ]:
from threading import Thread,Lock

def my_function(n,m): 
    with Lock():
        r = n/m
        print("Result is %s\n"%r)


def launch_thread(n,m):
    t = Thread(target = my_function, args = (n,m))
    t.start()
    return t
try:
    t1 = launch_thread(2,4)
    t2 = launch_thread(3,0)
    t3 = launch_thread(5,2)
    t1.join()
    t2.join()
    t3.join()
finally:
    print("done")

The 'with' clause can be used with lots of forms of context managers. For example, one way is just a class that defines __enter__() and __exit__(). 


In [ ]:
class SimpleCM():
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        print("Entering %s"%self.name)
        return self
        
    def __exit__(self, type, value, traceback):
        print("Exiting %s"%self.name)
        

with SimpleCM("Mytest") as m:
    print("Running stuff")

## contextlib
Python can make this even easier though if you wanted to do a decorator and take advantage of contextlib utilities

In [ ]:
from contextlib import contextmanager

@contextmanager
def simplercm(name):
    print("Entering %s"%name)
    yield
    print("Exiting %s"%name)
    
    
with simplercm("Mytest2") as m:
    print("Running stuff")    

Note ideally you want to catch any exceptions that might be thrown in the yield 

In [ ]:
from contextlib import contextmanager
@contextmanager
def simplercm(name):
    print("Entering %s"%name)
    try:
        yield
    finally:
        print("Exiting %s"%name)
    
    
with simplercm("Mytest2") as m:
    print("Running stuff")   
    
 

## Html Tag Example

In [ ]:
from contextlib import contextmanager

buffer = ""
@contextmanager
def tag(name):
    global buffer
    buffer += "<%s>" % name
    yield 
    buffer +="</%s>" % name
    
with tag("html"):
    with tag("header"):
        with tag("title"):
            buffer += "A title"
    with tag("body"):
        with tag("h1"):
            buffer += "Heading"
        with tag("p"):
            buffer += "This is a test"
        with tag("p"):
            buffer += "Another paragraph"
            with tag("b"):
                buffer += "bolded"
print(buffer)
        

## Login Example

Common header used by all

In [ ]:
from pymongo import MongoClient
import getpass
import requests
client = MongoClient('localhost', 27017)
db=client['test_database']
fac_collection = db['facilities']

Example 1: Where we started (no context managers)

In [ ]:
base_url = "http://foo.com"
password = getpass.getpass("Password: ")
r = requests.post(base_url+'/external_login',data={'username':'sysadmin','password':password,'facility_id':'System','format':'json'})
visit_id = r.cookies['cram-visit']
facilities = requests.get(base_url+'/facility/query?format=json&tg_visit='+visit_id).json()
for f in facilities:
    fac_collection.insert_one(f)
r = requests.post(base_url+'/external_logout',data={'tg_visit':visit_id})

fac_collection.count()

Notice the pair of login/logout operations. Easy enough to refactor....

Into a context manager and a client using it

Notice the exception block


In [ ]:
from contextlib import contextmanager
@contextmanager
def CramSession(base_url, username, password,facility_id):
    r = requests.post(base_url+'/external_login',data={'username':username,'password':password,'facility_id':facility_id,'format':'json'})
    visit_id = r.cookies['cram-visit']
    try:
        yield visit_id
    finally:
        r = requests.post(base_url+'/external_logout',data={'tg_visit':visit_id})
    

Client

In [ ]:

base_url = "http://foo.com"
with CramSession(base_url, "sysadmin", password, "System") as visit_id:
    facilities = requests.get(base_url+'/facility/query?format=json&tg_visit='+visit_id).json()
    for f in facilities:
        fac_collection.insert_one(f)
fac_collection.count()

While this is cleaner I really want something that feels a little 'cleaner' so decided to make a class 
that could operate as a context manager. Notice there is no need for an exception block

In [ ]:
class CramSession(object):
    def __init__(self, base_url, username, password, facility_id):
        self.base_url = base_url
        self.username = username
        self.password = password
        self.facility_id = facility_id
        self.visit_id = None
        
    def __enter__(self):
        r = requests.post(self.base_url+'/external_login',
                          data={'username':self.username,'password':self.password,'facility_id':self.facility_id,'format':'json'})
        self.visit_id = r.cookies['cram-visit']
        return self
        
    def __exit__(self, type, value, traceback):
        r = requests.post(self.base_url+'/external_logout',data={'tg_visit':self.visit_id})
        
    def all_facilities(self):
        return requests.get(self.base_url+'/facility/query?format=json&tg_visit='+self.visit_id).json()
    

Cient code is now fairly clean

In [ ]:
with CramSession("http://foo.com", "sysadmin", password, "System") as csession:
    for f in csession.all_facilities():
        fac_collection.insert_one(f)
fac_collection.count()
        

## DB Transaction Example

In this example what I want is a function that can be called in the midst of a transaction or in a new transaction. If it is already in a transaction then just keep adding to the existing transaction. If its a new transaction, then start a new session and when we are done close it. This can be referred to as a re-entrant context manager

class SessionWrapper(object):
    """
    Create a session wrapper which can close a session (if it created one)
    """
    def __init__(self, ds = None, session = None):
        """
        @param session: If None create a new session and close a session, otherwise don't
        """
        self.ds = ds
        self.session = session
        self._is_new = self.session is None

    def __enter__(self):
        if self._is_new:
            self.session = self.ds.create_session()
        return self.session

    def __exit__(self, exc_type, exc_value, traceback):
        if self._is_new:
            if self.session is not None:
                self.session.close()
                
    
def some_func(ds = None):    
    with SessionWrapper(ds=ds) as session:
        q = session.query(func.count(TraumaFact.c.PrimaryMechanismDesc), TraumaFact.c.PrimaryMechanismDesc)
        q = self.apply_mappings(mappings, q, parm_dict)
        q = q.group_by(TraumaFact.c.PrimaryMechanismDesc)
        data_list = self.execute_query(q)
    return (series, data_list, {})

## Convienence context managers

### closing

In [ ]:
class Simple(object):
    def __init__(self,s):
        self.s = s
        print("Opening %s"% self.s)

    def close(self):
        print ("Closing %s"%self.s)


In [ ]:
from contextlib import contextmanager

@contextmanager
def closing(e):
    try:
        yield e
    finally:
        e.close()
        
with closing(Simple("myt")) as mys:
    print("Doing whatever with %s"%mys.s)

In [ ]:
from contextlib import closing
with closing(Simple("myt")) as mys:
    print("Doing whatever with %s"%mys.s)

### Suppress

In [ ]:
from contextlib import suppress
def divide_zero(x,y):
    result = 0
    with suppress(ZeroDivisionError):
        result = x/y
    return result

In [ ]:
print(divide_zero(3,4))

In [ ]:
print(divide_zero(5,0))

### Redirections

In [ ]:
import io
from contextlib import redirect_stdout
f = io.StringIO()
with redirect_stdout(f):
    help(dir)
print(f.getvalue()[:80])


# A Timely Example
## A Context Manager


In [ ]:
from contextlib import contextmanager
from time import time
import numpy as np

@contextmanager
def VersaTimer():
    """A robust time example blatently reused"""
    start = time()
    print("Staring at {}".format(start))
    try:
        yield
    finally:
        end = time()
        print("Ending at {} (total: {})".format(end, end - start))
        

def make_l_array(n, m):
    p_array = [x for x in range(n)]
    return [x*m for x in p_array]

def make_l_array2(n, m):    
    n_array = np.arange(n)
    return n_array*m

    
    
with VersaTimer():
    n_array = make_l_array(100000,5)
    
with VersaTimer():
    n_array2 = make_l_array2(100000,5)
assert(len(n_array)) == len(n_array2)



## A decorator 

In [ ]:
from time import time
from functools import wraps
def mytimer_wrapper(wrapped):
    @wraps(wrapped)
    def inner(*args,**kwargs):
        start = time()
        print("Staring at {}".format(start))
        try:
            retval = wrapped(*args, **kwargs)
        finally:
            end = time()
        print("Ending at {} (total: {})".format(end, end - start))
    return inner

@mytimer_wrapper
def sum_power(n,p=2):
    sum = 0
    for i in range(n):
        sum += pow(i,p)
    return sum
    

print("Sum ^2 is {}".format(sum_power(1000000)))
    
print("Sum ^8 is {}".format(sum_power(1000000,8)))

So now we can see doing it as a decorator OR a context library.

What about 
## Both a decorator and a context manager ...


In [ ]:
from contextlib import ContextDecorator
from time import time

class VersaTimer(ContextDecorator):

    def __enter__(self):
        self.start = time()
        print("Starting at {}".format(self.start))
        return self

    def __exit__(self, type, value, traceback):
        self.end = time()
        total = self.end - self.start
        print("Ending at {} (total: {})".format(self.end, total))
        
def sum_power(n,p=2):
    sum = 0
    for i in range(n):
        sum += pow(i,p)
    return sum
    
@VersaTimer()
def mult_power(n,p=2):
    mult = 1
    for i in range(1,n):
        mult *= pow(i,p)
    return mult

with VersaTimer():
    print("Sum ^2 is {}".format(sum_power(1000000)))
    
with VersaTimer():
    print("Sum ^8 is {}".format(sum_power(1000000,8)))


print("Mult ^2 is {}".format(mult_power(100,2)))

